In [57]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import os

def file_txt_to_sql(og_filename, filename, table):
    chunk_size = 10_000
    og_file = f'/Users/mmunozcampos/Documents/Comp y reg/{og_filename}'
    db_path = f"/Users/mmunozcampos/Documents/Comp y reg/{filename}"
    
    # Nombres de las columnas
    column_names = [
        'codigo_aseguradora','periodo_informacion', 'tipo_registro', 'run_beneficiario',
        'sexo_beneficiario', 'edad_beneficiario', 'tipo_beneficiario', 'rut_prestador',
        'region_prestador', 'prestador_preferente', 'programa_medico_principal',
        'programa_medico_complementario', 'bono_atencion', 'reembolso',
        'codigo_prestacion', 'pertenencia_codigo',
        'cobertura_financiamiento_prestacion', 'fecha_bonificacion',
        'tipo_prestador', 'tipo_atencion', 'frecuencia', 'valor_facturado',
        'valor_bonificado', 'monto_copago', 'bonificacion_restringida',
        'tipo_plan', 'modalidad_intervencion_quirurgica',
        'ley_urgencia_vital', 'codigo_gpp', 'identificacion_ges', 'none'
    ]
    
    # Creamos SQLite engine con extended timeout y pragma settings
    engine = create_engine(f"sqlite:///{db_path}", 
                          connect_args={"timeout": 300, "check_same_thread": False})
    
    # Prevención de problemas de tipeo
    conn = sqlite3.connect(db_path)
    conn.execute("PRAGMA journal_mode = WAL;")  # Write-Ahead Logging for better performance
    conn.execute("PRAGMA synchronous = NORMAL;")  # Less durability but faster performance
    conn.close()
    
    cols_numericas = [
        'codigo_aseguradora','run_beneficiario', 'rut_prestador',
        'region_prestador', 'programa_medico_principal',
        'programa_medico_complementario', 'bono_atencion', 'reembolso',
        'frecuencia', 'valor_facturado',
        'valor_bonificado', 'monto_copago'
    ]

    #preparamos el porcentaje de completado
    total_lines = sum(1 for _ in open(og_file, encoding='latin1'))
    total_chunks = total_lines // chunk_size + 1

    #preparamos el reporte del progreso
    last_reported = -1
    
    # Correr en chunks 
    for i, chunk in enumerate(pd.read_csv(og_file, delimiter='|', 
                                          names=column_names, 
                                          header=None, 
                                          chunksize=chunk_size, 
                                          encoding='latin1' 
                                         )):
        #Nos encargamos de las columnas numericas
        for col in cols_numericas:
            if 'object' in str(chunk[col].dtype):
                chunk[col] = chunk[col].str.strip()
                chunk[col] = pd.to_numeric(chunk[col], errors='coerce')
        #exportamos a sql
        chunk.iloc[:, :-1].to_sql(table, con=engine, if_exists='append', index=False)

        percent_complete = (i + 1) / total_chunks * 100
        current_report = int(percent_complete // 5)

        # Reporta cada 5% de aumento
        if current_report > last_reported:
            print(f"Cargando: {current_report * 5}% completo")
            last_reported = current_report
        #para testeo
        #if i == 0:
        #    break
        
    engine.dispose()
    return print(f'{filename} fue un exito')

#### Cargamos las bases

In [58]:
file_txt_to_sql("Prestaciones_Bonificadas_2024t1.txt", "prestaciones_t2024.db", "tabla2024")

Cargando: 5% completo
Cargando: 10% completo
Cargando: 15% completo
Cargando: 20% completo
Cargando: 25% completo
Cargando: 30% completo
Cargando: 35% completo
Cargando: 40% completo
Cargando: 45% completo
Cargando: 50% completo
Cargando: 55% completo
Cargando: 60% completo
Cargando: 65% completo
Cargando: 70% completo
Cargando: 75% completo
Cargando: 80% completo
Cargando: 85% completo
Cargando: 90% completo
Cargando: 95% completo
Cargando: 100% completo
prestaciones_t2024.db fue un exito


In [59]:
file_txt_to_sql("Prestaciones_Bonificadas_2024t2.txt", "prestaciones_t2024.db", "tabla2024")

Cargando: 5% completo
Cargando: 10% completo
Cargando: 15% completo
Cargando: 20% completo
Cargando: 25% completo
Cargando: 30% completo
Cargando: 35% completo
Cargando: 40% completo
Cargando: 45% completo
Cargando: 50% completo
Cargando: 55% completo
Cargando: 60% completo
Cargando: 65% completo
Cargando: 70% completo
Cargando: 75% completo
Cargando: 80% completo
Cargando: 85% completo
Cargando: 90% completo
Cargando: 95% completo
Cargando: 100% completo
prestaciones_t2024.db fue un exito


In [60]:
file_txt_to_sql("Prestaciones_Bonificadas_2024t3.txt", "prestaciones_t2024.db", "tabla2024")

Cargando: 5% completo
Cargando: 10% completo
Cargando: 15% completo
Cargando: 20% completo
Cargando: 25% completo
Cargando: 30% completo
Cargando: 35% completo
Cargando: 40% completo
Cargando: 45% completo
Cargando: 50% completo
Cargando: 55% completo
Cargando: 60% completo
Cargando: 65% completo
Cargando: 70% completo
Cargando: 75% completo
Cargando: 80% completo
Cargando: 85% completo
Cargando: 90% completo
Cargando: 95% completo
Cargando: 100% completo
prestaciones_t2024.db fue un exito


In [61]:
file_txt_to_sql("Prestaciones_Bonificadas_2024t4.txt", "prestaciones_t2024.db", "tabla2024")

Cargando: 5% completo
Cargando: 10% completo
Cargando: 15% completo
Cargando: 20% completo
Cargando: 25% completo
Cargando: 30% completo
Cargando: 35% completo
Cargando: 40% completo
Cargando: 45% completo
Cargando: 50% completo
Cargando: 55% completo
Cargando: 60% completo
Cargando: 65% completo
Cargando: 70% completo
Cargando: 75% completo
Cargando: 80% completo
Cargando: 85% completo
Cargando: 90% completo
Cargando: 95% completo
Cargando: 100% completo
prestaciones_t2024.db fue un exito


In [62]:
import sqlite3
import pandas as pd

db_path = "/Users/mmunozcampos/Documents/Comp y reg/prestaciones_t2024.db"
conn = sqlite3.connect(db_path)

#### Identifique los ocho principales prestadores hospitalarios en la RM según la cantidad de prestaciones bonificadas.

In [110]:
query = ''' SELECT rut_prestador, SUM(frecuencia) AS frecuencia_total
        FROM tabla2024 
        WHERE tipo_atencion == "Atención Hospitalaria" AND region_prestador == 13
        GROUP BY rut_prestador
        ORDER BY frecuencia_total DESC
        LIMIT 8; '''
		
top8_rm = pd.read_sql(query, conn)
top8_rm

,rut_prestador,frecuencia_total
0,628513,2263154
1,1324822,1745725
2,3325777,1558533
3,1159334,1331688
4,2868186,1117942
5,422649,813265
6,27564,697531
7,922214,561771


In [113]:
#Exportamos a excel
top8_rm.to_excel('top8_rm.xlsx', index=False)

#### Calcule la participación de mercado de cada uno.

In [116]:
query = ''' SELECT 
    rut_prestador,
    SUM(frecuencia) * 100.0 / (
        SELECT SUM(frecuencia)
        FROM tabla2024
        WHERE region_prestador = 13
          AND tipo_atencion = "Atención Hospitalaria"
    ) AS market_share_percentage
FROM tabla2024
WHERE region_prestador = 13
  AND tipo_atencion = "Atención Hospitalaria"
GROUP BY rut_prestador
ORDER BY market_share_percentage DESC
LIMIT 8;

'''
top8_marketshare = pd.read_sql(query, conn)

In [65]:
top8_marketshare["market_share_percentage"] = top8_marketshare["market_share_percentage"].round(2)
top8_marketshare

,rut_prestador,market_share_percentage
0,628513,15.90
1,1324822,12.27
2,3325777,10.95
3,1159334,9.36
4,2868186,7.85
5,422649,5.71
6,27564,4.90
7,922214,3.95


In [109]:
#Exportamos a excel
top8_marketshare.to_excel('top8_market_share.xlsx', index=False)

#### Para los principales cinco prestadores identificados, determine cuál es la aseguradora con mayor volumen de prestaciones bonificadas.

In [151]:
query = ''' SELECT rut_prestador, codigo_aseguradora, SUM(frecuencia) AS frecuencia_total
    FROM tabla2024
    WHERE tipo_atencion = "Atención Hospitalaria"
    AND region_prestador = 13
    AND rut_prestador IN (628513, 1324822, 3325777, 1159334, 2868186)
    GROUP BY rut_prestador, codigo_aseguradora
    ORDER BY
        CASE rut_prestador
            WHEN 628513 THEN 1
            WHEN 1324822 THEN 2
            WHEN 3325777 THEN 3
            WHEN 1159334 THEN 4
            WHEN 2868186 THEN 5
        END;
    '''
df_share_aseguradora = pd.read_sql(query, conn)

In [152]:
# Opcional por si se quieren ver todos los valores:
# df_share_aseguradora

#### Calcule el porcentaje que representa dicha aseguradora respecto del total de prestaciones bonificadas para ese prestador.

In [153]:
#creamos la columna del total para cada prestador
df_share_aseguradora['sum_frecuencia'] = df_share_aseguradora.groupby('rut_prestador')['frecuencia_total'].transform('sum')

In [154]:
#creamos el share de cada asegurador
df_share_aseguradora["share_aseguradora"] = df_share_aseguradora["frecuencia_total"]*100/df_share_aseguradora["sum_frecuencia"]
df_share_aseguradora["share_aseguradora"] = df_share_aseguradora["share_aseguradora"].round(2)

In [156]:
#Mantenemos solo la máxima aseguradora
df_max_share_asg = df_share_aseguradora.loc[df_share_aseguradora.groupby('rut_prestador')['share_aseguradora'].idxmax()]

In [159]:
df_max_share_asg.drop('sum_frecuencia', axis=1, inplace=True)

In [161]:
df_max_share_asg

,rut_prestador,codigo_aseguradora,frecuencia_total,share_aseguradora
1,628513,67,1608351,71.07
32,1159334,99,441737,33.17
10,1324822,67,1105119,63.30
35,2868186,67,475821,42.56
24,3325777,99,790250,50.70


In [162]:
#Exportamos a excel
df_max_share_asg.to_excel('top5_aseguradoras.xlsx', index=False)

#### Dentro de la Región Metropolitana, identifique los cinco procedimientos hospitalarios más comunes asociados al Grupo 11 (según el arancel MLE).

In [63]:
query = ''' SELECT codigo_prestacion, SUM(frecuencia) AS frecuencia_total
        FROM tabla2024
        WHERE tipo_atencion == "Atención Hospitalaria"
        AND region_prestador = 13
        AND substr(codigo_prestacion, 1, 2) = '11'
        AND length(codigo_prestacion) > 3
        GROUP BY codigo_prestacion
        ORDER BY frecuencia_total DESC
        LIMIT 10; '''
top5_neuro = pd.read_sql(query, conn)
top5_neuro

,codigo_prestacion,frecuencia_total
0,1101011,6155
1,1101050,5500
2,1103048,4209
3,1101043,3586
4,1101045,3514
5,1103049,3188
6,1101003,2101
7,1103069,1688
8,1103066,1442
9,1101006,1326


Colocamos más en caso de que no aparezcan en la base de fonasa.

In [71]:
df_fonasa = pd.read_excel("Códigos Fonasa.xlsx")

In [78]:
df_fonasa[df_fonasa["grupo"] == 11].head(4)

,código,código (4),código (c/guarismo) para merge,grupo,sub_grupo,presta,equipo,pab,Num,glosa,Especialidad,Sub especialidad
859,1101001,1101,110100100,11,1,1,0,0,0.0,RELLENADO DE BOMBAS DE ADMINISTRACION DE FARM...,Neurología y neurocirugía,NaN
860,1101002,1101,110100200,11,1,2,0,0,0.0,PUNCION SUBDURAL ...,Neurología y neurocirugía,NaN
861,1101003,1101,110100300,11,1,3,0,0,0.0,PUNCION LUMBAR C/S MANOMETRIA C/S QUECKENSTED ...,Neurología y neurocirugía,NaN
862,1101140,1101,110114000,11,1,140,0,0,0.0,"ESCLEROSIS MULTIPLE REMITENTE RECURRENTE, TRA...",Neurología y neurocirugía,NaN


In [79]:
df_fonasa[df_fonasa["grupo"] == 11].tail(4)

,código,código (4),código (c/guarismo) para merge,grupo,sub_grupo,presta,equipo,pab,Num,glosa,Especialidad,Sub especialidad
962,1103065,1103,110306508,11,3,65,2,8,8.0,"SINDROME DE COSTILLA CERVICAL, TRAT. QUIR. ...",Neurología y neurocirugía,NaN
963,1103066,1103,110306607,11,3,66,2,7,7.0,LIBERACION QUIRURGICA DE NERVIO PERIFERICO EXT...,Neurología y neurocirugía,NaN
964,1103067,1103,110306707,11,3,67,2,7,7.0,"LIBERACION DE NERVIO CUBITAL A NIVEL DEL CODO,...",Neurología y neurocirugía,NaN
965,1103083,1103,110308304,11,3,83,2,4,4.0,LIBERACION QUIRURGICA DE NERVIO PERIFERICO EN...,Neurología y neurocirugía,NaN


Como vemos, todos los códigos parten por 11, eso lo usaremos para filtrar la base.

In [106]:
top_op = pd.DataFrame({
    'codigo': [],
    'frecuencia': [],
    'glosa': []
})
for i, cod in enumerate(top5_neuro["codigo_prestacion"].unique()):
    if cod in df_fonasa["código"].unique():
        row = top5_neuro[top5_neuro["codigo_prestacion"]==cod].index[0]
        freq = top5_neuro.iloc[row]["frecuencia_total"]
        glosa = df_fonasa[df_fonasa["código"] == cod]["glosa"].iloc[0]
        
        new_row = {'codigo': cod, 'frecuencia': freq, 'glosa': glosa}
        top_op = pd.concat([top_op, pd.DataFrame([new_row])], ignore_index=True)

top_op['codigo'] = top_op['codigo'].astype(int)
top_op['frecuencia'] = top_op['frecuencia'].astype(int)


In [114]:
top_op = top_op.head(5)

In [115]:
#Exportamos a excel
top_op.to_excel('top5_prod_hosp.xlsx', index=False)

In [ ]:
conn.close()